In [3]:
import pandas as pd

file_txt = open('wr112006.txt', 'r')
file_csv = open('dataset.csv', 'w')
for line in file_txt:
    stroka = line.replace(';', ',').replace(' ', '')
    file_csv.write(stroka)



In [67]:
columns = ['station', 'year', 'month', 'day', 'summary', 'min_temperature', 'avarage_temperature', 'max_temperature', 'precipitation']
df = pd.read_csv('dataset.csv', names = columns)


In [95]:
df_filtered = df[(df['month'] > 4) & (df['month'] < 10)]

print(df_filtered)

         station  year  month  day  summary  min_temperature  \
120      20046.0  1990      5    1        0            -17.9   
121      20046.0  1990      5    2        0            -17.5   
122      20046.0  1990      5    3        0            -18.1   
123      20046.0  1990      5    4        0            -13.7   
124      20046.0  1990      5    5        0            -16.0   
...          ...   ...    ...  ...      ...              ...   
5770951  37663.0  2020      9   26        0             10.0   
5770952  37663.0  2020      9   27        0             11.6   
5770953  37663.0  2020      9   28        0             11.3   
5770954  37663.0  2020      9   29        0              9.1   
5770955  37663.0  2020      9   30        0              5.3   

         avarage_temperature  max_temperature  precipitation  
120                    -16.7            -12.5            0.0  
121                    -15.3            -12.9            0.0  
122                    -14.9            -1

In [104]:
df_droped = df_filtered.drop(columns=['summary', 'min_temperature', 'precipitation'], axis=1)

df_droped.dropna(inplace=True)
df_rdf_droped['station'].astype(int)

print(df_droped)


         station  year  month  day  avarage_temperature  max_temperature
120      20046.0  1990      5    1                -16.7            -12.5
121      20046.0  1990      5    2                -15.3            -12.9
122      20046.0  1990      5    3                -14.9            -12.6
123      20046.0  1990      5    4                -12.6             -9.5
124      20046.0  1990      5    5                -13.2            -11.0
...          ...   ...    ...  ...                  ...              ...
5770951  37663.0  2020      9   26                 14.7             21.1
5770952  37663.0  2020      9   27                 13.8             16.6
5770953  37663.0  2020      9   28                 12.8             15.7
5770954  37663.0  2020      9   29                 13.1             19.4
5770955  37663.0  2020      9   30                 10.1             17.3

[2376759 rows x 6 columns]
